In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

sid_dir = '/Volumes/sim/data/OSISAF_ice_drift_CDR_postproc'
mesh_dir = '/Volumes/sim/data/NERSC_arctic25km_sea_ice_age_v2p1/mesh/'
unc_dir = '/Volumes/sim/data/NERSC_arctic25km_sea_ice_age_v2p1/unc/'
sic_dir = '/Volumes/sim/data/NERSC_arctic25km_sea_ice_age_v2p1/sic/'


In [ ]:
sid_files = sorted(glob.glob(f'{sid_dir}/199[1-7]/ice_drift*npz'))
len(sid_files)

In [ ]:
sic2d_avgs = []
sid2d_avgs = []
sic2d_unc_avgs = []
sid2d_unc_avgs = []
dates2d = []

for sid_file in tqdm(sid_files[::10]):
    with np.load(sid_file) as f:
        sic_unc = f['sic_unc'][1:-1:3, 1:-1:3]
        sic = f['c'][1:-1:3, 1:-1:3]
        sid_unc = f['sid_unc']
        sid = np.hypot(f['u'], f['v'])

    sic2d_avgs.append(np.nanmean(sic[sic > 15]))
    sid2d_avgs.append(np.nanmean(sid[sic > 15]))
    sic2d_unc_avgs.append(np.nanmean(sic_unc[sic > 15]))
    sid2d_unc_avgs.append(np.nanmean(sid_unc[sic > 15]))
    dates2d.append(sid_file.split('.')[0].split('-')[-1])

In [ ]:
pd_dates2d = pd.to_datetime(dates2d)
sic2d_avgs = np.array(sic2d_avgs)
sid2d_avgs = np.array(sid2d_avgs)
sic2d_unc_avgs = np.array(sic2d_unc_avgs)
sid2d_unc_avgs = np.array(sid2d_unc_avgs)

In [ ]:
unc_sic_files = sorted(glob.glob(f'{unc_dir}/1991/unc_sic_*.npz'))
unc_sid_files = sorted(glob.glob(f'{unc_dir}/1991/unc_sid_*.npz'))
unc_sidsic_files = sorted(glob.glob(f'{unc_dir}/1991/unc_sidsic_*.npz'))

len(unc_sic_files[10:]), len(unc_sid_files[10:]), len(unc_sidsic_files)

In [ ]:
sic_avgs = []
unc_sic_avgs = []
unc_sid_avgs = []
unc_sidsic_avgs = []

dates = []

for unc_sic_file, unc_sid_file, unc_sidsic_file in zip(unc_sic_files[10::10], unc_sid_files[10::10], unc_sidsic_files[::10]):
    with np.load(unc_sic_file) as data:
        sic_min = data['sic_min']
        unc_sic = data['unc_sic']
    with np.load(unc_sid_file) as data:
        unc_sid = data['unc_sid']
    with np.load(unc_sidsic_file) as data:
        unc_sidsic = data['unc_sic_sid']
    unc_date = unc_sic_file.split('_')[-1].split('.')[0]
    dates.append(unc_date)
    sic_avgs.append(np.mean(sic_min[sic_min > 15]))
    unc_sic_avgs.append(np.mean(unc_sic[sic_min > 15]))
    unc_sid_avgs.append(np.mean(unc_sid[sic_min > 15]))
    unc_sidsic_avgs.append(np.mean(unc_sidsic[sic_min > 15]))
    #mesh_file = f'{mesh_dir}/1991/mesh_{unc_date}.npz'
    ##with np.load(mesh_file) as data:
    #    x = data['x']
    #    y = data['y']
    #    t = data['t']
    #plt.tripcolor(x, y, t, unc_sic, cmap='viridis')
    #plt.tripcolor(x, y, t, sic_min, cmap='viridis')
    #break

In [ ]:
pd_dates = pd.to_datetime(dates)
sic_avgs = np.array(sic_avgs)
unc_sic_avgs = np.array(unc_sic_avgs)
unc_sid_avgs = np.array(unc_sid_avgs)
unc_sidsic_avgs = np.array(unc_sidsic_avgs)
unc_tot = np.hypot(unc_sic_avgs, unc_sidsic_avgs)

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(5, 8), sharex=True)
line0 = axs[0].plot(pd_dates2d, sic2d_unc_avgs)[0]
axs01 = axs[0].twinx()
line01 = axs01.plot(pd_dates2d, sic2d_avgs, color='C1', alpha=0.8, lw=1)[0]


axs[0].set_ylabel('$\\sigma_{OBS}$, %')
axs01.set_ylabel('$C_{OBS}$, %', color='C1')
axs[0].set_title('A) Observed SIC and uncertainty')

axs[1].plot(pd_dates2d, sid2d_unc_avgs)[0]
axs11 = axs[1].twinx()
axs11.plot(pd_dates2d, sid2d_avgs, color='C1', alpha=0.8, lw=1)[0]

axs[1].set_ylabel('$\\sigma_S$, km d$^{-1}$')
axs11.set_ylabel('Drift, km d$^{-1}$', color='C1')
axs[1].set_title('B) Observed SID and uncertainty')

line2 = axs[2].plot(pd_dates, unc_sic_avgs, color='C0')[0]
axs21 = axs[2].twinx()
line21 = axs21.plot(pd_dates, sic_avgs, color='C1', alpha=0.9, lw=1)[0]

axs[2].set_ylabel('$\\sigma_{MIN}$, %')
axs21.set_ylabel('$C_A$, %', color='C1')
axs[2].set_title('C) Conc. and uncertainty of advected field')

line3 = axs[3].plot(pd_dates, unc_sid_avgs, color='C0')[0]
axs[3].set_ylabel('$\\sigma_I$, km')
axs[3].set_title('D) Integrated SID uncertainty')

line4 = axs[4].plot(pd_dates, unc_sidsic_avgs, color='C0', label='$\\sigma_{SID}$')[0]
line42 = axs[4].plot(pd_dates, unc_tot, color='C1', label='$\\sigma_{COM}$')[0]
axs[4].set_ylabel('$\\sigma_{SID}$, $\\sigma_{COM}$, %')
axs[4].legend()
axs[4].set_title('E) Combined uncertainty')

for ax in axs:
    for date in ['1991-09-15', '1992-09-15', '1993-09-15', '1994-09-15', '1995-09-15', '1996-09-15', '1997-09-15']:
        ax.axvline(pd.to_datetime(date), color='r', linestyle='--')

axs[0].set_xlim([pd.to_datetime('1991-09-10'), pd.to_datetime('1997-09-20')])
plt.tight_layout()
plt.savefig('sic_sid_uncertainty_timeseries.png', dpi=200, pad_inches=0.1)
